In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objs as graph_obj
import dash
import dash_core_components as dcc
import dash_html_components as dhc

In [2]:
r_inv = 3
r_inc = 30
r_cap = 20.315
age = 30

In [3]:
def calc_rate(r_inv, r_inc, r_cap, amt, n):
    r_0 = np.power(1.0 + r_inv, n)
    r_1 = np.power(1.0 + r_inv, n - 1)
    r_s = r_0 - np.max([r_0 - 1.0, 0.0]) * r_cap
    r_i = r_0 + (r_1 - np.max([r_1 - 1.0, 0.0]) * r_cap) * r_inc
    ret = np.power(r_i / r_s, 1.0 / n) - 1.0
    delta = amt * (r_i - r_s)
    return ret, delta

In [4]:
def gen_seq(r_inv, r_inc, r_cap, amt, N):
    x = []
    r = []
    a = []
    for n in range(N, 60):
        res = calc_rate(r_inv, r_inc, r_cap, amt, 60 - n)
        x.append(n)
        r.append(res[0] * 100.0)
        a.append(int(res[1]))
    return x, r, a

In [ ]:
app = dash.Dash()
app.layout = dhc.Div([
    dhc.Div([
        dhc.Div('投資収益率 (%)'),
        dcc.Input(
            id='r_inv',
            value=r_inv,
            type='number', inputmode='numeric'
        ),
    ]),
    dhc.Div([
        dhc.Div('所得税率 + 住民税率 (%)'),
        dcc.Input(
            id='r_inc',
            value=r_inc,
            type='number', inputmode='numeric'
        ),
    ]),
    dhc.Div([
        dhc.Div('譲渡税率 (%)'),
        dcc.Input(
            id='r_cap',
            value=r_cap,
            type='number', inputmode='numeric'
        ),
    ]),
    dhc.Div([
        dhc.Div('拠出開始時の年齢'),
        dcc.Input(
            id='age',
            value=age,
            type='number', inputmode='numeric'),
    ]),
    dhc.Div([
        dhc.Div('毎月の掛け金'),
        dcc.Input(
            id='amt',
            value=23000,
            type='number', inputmode='numeric'),
    ]),
    dcc.Graph(
        id='graph1',
        figure={},
        style={}
    )
])
@app.callback(
    dash.dependencies.Output(component_id='graph1', component_property='figure'),
    [
        dash.dependencies.Input(component_id='r_inv', component_property='value'),
        dash.dependencies.Input(component_id='r_inc', component_property='value'),
        dash.dependencies.Input(component_id='r_cap', component_property='value'),
        dash.dependencies.Input(component_id='age', component_property='value'),
        dash.dependencies.Input(component_id='amt', component_property='value'),
    ],
)
def update_output(input_r_inv, input_r_inc, input_r_cap, input_age, input_amt):
    data = []
    try:
        r_inv = float(input_r_inv) / 100.0
        r_inc = float(input_r_inc) / 100.0
        r_cap = float(input_r_cap) / 100.0
        age = int(input_age)
        amt = 12 * int(input_amt)
    except ValueError:
        age = 60
        a = 0
    if age < 60:
        x, r, a = gen_seq(r_inv, r_inc, r_cap, amt, age)
        trace_a = graph_obj.Bar(
            x=x, y=a,
            name='将来価値'
        )
        data.append(trace_a)
        trace_r = graph_obj.Scatter(
            x=x, y=r,
            name='収益率',
            yaxis='y2'
        )
        data.append(trace_r)
    total = int(np.sum(a) / 10000)
    layout = {
        'title': f"iDeCo利用によるお得分の評価 - 将来価値総額: {total}万円",
        'xaxis': {'title': '拠出時年齢'},
        'yaxis': {
            'title': '将来価値',
            'rangemode': 'tozero',
            'showgrid': False,
        },
        'yaxis2': {
            'title': '収益率',
            'rangemode': 'tozero',
            'showgrid': False,
            'overlaying': 'y',
            'side': 'right',
        },
    }
    return {
        'data': data,
        'layout': layout,
    }

In [ ]:
app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
